In [ ]:
import numpy as np
import pandas as pd
import time
import os
import pathlib
import copy

import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras import losses, optimizers
from tensorflow.keras.models import Sequential, load_model
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Input, Activation, Dense, TimeDistributed,Conv2D,MaxPooling2D, Lambda,UpSampling2D
from tensorflow.keras.layers import add, dot, concatenate, LSTM, Bidirectional,Reshape,Flatten
import tensorflow.keras as keras
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
#from tf.compat.v1.keras.layers.CuDNNLSTM
from Utils.PlotLosses import PlotLosses
from Utils.Utils import dist_coordinates, TrainModel, s_DataFrame, s_relative_and_std, cal_accuracy, euclidean_distance_loss,set_input,s_labels, change_relative_coordinates, get_reculsive_error, re_coordinates
from Utils.Utils import get_reculsive_error_m3_diff_timestep, get_reculsive_error_m123, get_reculsive_error_m3_only_nearest,cal_accuracy_tmp, dist_euclidean
path = './Data_v2.0/7Robots/TestSet'
data_test = pd.read_csv(path+'/Test.csv')

std_coordinates = 500
std_orientation = 180

robots = ['H','F1','F2','F3','F4','F5','T']
std_robot = 'T'
coordinates = ['x','y']
orientation = ['o']
_n_hist = 10
_n_samples = 26

In [ ]:
K.clear_session()

## Load model

In [ ]:
Model_hist = list()
_n = 8
for i in range(0,10) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model1_hist_{}_{}.h'.format(_n,i), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model_hist.append(_model)

In [ ]:
## concatenates pred_result and original data
## Change std_robot and change relative coordinates 
## calculate error

## Reculsive Error

### Model1

In [ ]:
#_obs_list = [['_F3_x','_F3_y','_T_o'],['_F2_x','_F2_y','_F3_o'],['_F1_x','_F1_y','_F2_o']]
#pred_list = [['_F2_x','_F2_y','_F3_o','_F2_o'],['_F1_x','_F1_y','_F2_o','_F1_o'],['_H_x','_H_y','_F1_o','_H_o']]
#robots = ['H','F1','F2','F3','T']
std_coordinates = 260
std_orientation = 180

_obs_list = [['_F5_x','_F5_y'],['_F4_x','_F4_y'],
             ['_F3_x','_F3_y'],['_F2_x','_F2_y'],
             ['_F1_x','_F1_y']]
pred_list = [['_F4_x','_F4_y','_F5_o','_F4_o'],['_F3_x','_F3_y','_F4_o','_F3_o'],
            ['_F2_x','_F2_y','_F3_o','_F2_o'],['_F1_x','_F1_y','_F2_o','_F1_o'],
            ['_H_x','_H_y','_F1_o','_H_o']]
robots = ['H','F1','F2','F3','F4','F5','T']

In [ ]:
_errs = list()
_results = list()
for i in range(0,len(Model_hist)) :
    _result = get_reculsive_error(_Model=Model_hist[i],start = _n_hist-_n,_n_model = 1,std_robot='T',_obs_list=_obs_list,pred_list=pred_list,robots=['H','F1','F2','F3','F4','F5','T'],data_test = data_test,_n_samples=_n_samples,_n_hist=_n_hist,std_coordinates=std_coordinates)
    _error, _pred_result = cal_accuracy_tmp(data=_result,model_n = 1,data_test=data_test,std_coordinates=std_coordinates, _n_samples=_n_samples)
    _errs.append(_error)
    _results.append(_pred_result)
    print('Model {} complete'.format(i))

## Save Results

In [ ]:
for i in range(0,len(_results)) :
#    _err1[i].to_csv('Error_1_{}.csv'.format(i))
    _results[i].to_csv('Results/Diff_History_{}_{}_Predict_v2.csv'.format(_n,i))    